In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.applications.resnet50 import ResNet50
import keras
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.6.0'

# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `KL0000000001,B,B[pq],W[dd],B[dp],W[pd],B[jc],...`. 

Columns are:

    1. KL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [3]:
df = open('./Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [1]:
#0:所有黑棋
#1:所有白棋
#2:標示空地
#3~10:最後8步
#11:周圍黑棋7*7
#12:周圍白棋7*7
def prepare_input(moves):
    x = np.zeros((19,19,13))
    if len(moves) == 0:
        return x
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    
    #倒數8步
    sz = len(moves)
    last1 = sz - 1
    last8 = max(sz - 9, 0)
    for i in range(last1, last8, -1):
        col = coordinates[moves[i][2]]
        row = coordinates[moves[i][3]]
        x[row, col, 11 - (sz - i)] = 1
    
    #周圍7*7
    last_col = coordinates[moves[-1][2]]
    last_row = coordinates[moves[-1][3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 11] = x[i, j, 1]
            x[i, j, 12] = x[i, j, 2]
            
    #列印所有棋盤
#     for i in range(0, 13, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x
def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [6]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [63]:
def data_generator(games, batch_size):
    def generator():
        x_batch = [] # Initialize data batch
        y_batch = [] # Initialize target batch
        for game_i, game in enumerate(games): # Iterate through games
            moves_list = game.split(',')
#             print(moves_list)
            for count, move in enumerate(moves_list):
                if colors[game_i] == move[0]:
#                     print(move)
                    x_batch.append(prepare_input(moves_list[:count]))
                    y_batch.append(prepare_label(moves_list[count]))
                    if len(x_batch) == batch_size: # Yield when reached batch size
                        yield np.array(x_batch), tf.one_hot(np.array(y_batch), depth=19*19)
                        x_batch = []
                        y_batch = []
    return generator

batch_size = 128
val_rate = 0.1
split_point = int(len(games) * (1 - val_rate))
generator = data_generator(games[:split_point], batch_size)
dataset = tf.data.Dataset.from_generator(generator, 
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,13)),tf.TensorShape((batch_size,361)))
                                        )
# SHUFFLE_BUFFER_SIZE = 200
# dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [59]:
val_generator = data_generator(games[:split_point], batch_size)
val_dataset = tf.data.Dataset.from_generator(val_generator, 
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,13)),tf.TensorShape((batch_size,361)))
                                        )
# SHUFFLE_BUFFER_SIZE = 200
# dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
val_dataset = dataset.prefetch(tf.data.AUTOTUNE)

# Training

### Simple DCNN Model:

In [60]:
def create_model():
    inputs = Input(shape=(19, 19, 13))
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Nadam(learning_rate = 0.0005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [61]:
model = create_model()
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 19, 19, 13)]      0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 19, 19, 64)        40832     
_________________________________________________________________
batch_normalization_108 (Bat (None, 19, 19, 64)        256       
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 19, 19, 64)        200768    
_________________________________________________________________
batch_normalization_109 (Bat (None, 19, 19, 64)        256       
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 19, 19, 64)        102464    
_________________________________________________________________
batch_normalization_110 (Bat (None, 19, 19, 64)        256 

In [10]:
def Conv2D_BN(inputs,growth_rate):
    nfilter = growth_rate * 4  
    outputs  = keras.layers.Activation('relu')(inputs)
    outputs  = keras.layers.BatchNormalization()(outputs)
    outputs = keras.layers.Conv2D(filters = nfilter,kernel_size = (1,1),padding='same',strides=1)(outputs)
    outputs  = keras.layers.Activation('relu')(outputs)
    outputs  = keras.layers.BatchNormalization()(outputs)
    outputs = keras.layers.Conv2D(filters = growth_rate,kernel_size=(3,3),padding='same',strides=1)(outputs)
    return outputs

def dense_block(inputs,growth_rate,n_filter,layers):
    concat = inputs 
    for i in range(layers):
        outputs = Conv2D_BN(concat,growth_rate)
        concat = keras.layers.concatenate([outputs,concat])
        n_filter += growth_rate
    return concat,n_filter

def transition_block(inputs,n_filter):
    print(n_filter)
    nfilter = int(n_filter*0.5)
    outputs  = keras.layers.Activation('relu')(inputs)
    outputs  = keras.layers.BatchNormalization()(outputs)
    outputs = keras.layers.Conv2D(filters = nfilter,kernel_size = (1,1),padding='same',strides=1)(outputs )
#     outputs = keras.layers.AveragePooling2D(pool_size=2)(outputs)
    return outputs,nfilter

def get_model():
    nfilter = 32
    growth_rate = 32
    inputs = keras.Input(shape=(19,19, 11))
    x = keras.layers.Conv2D(filters = nfilter,kernel_size=(7,7),padding='same',strides=1,activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
#     x = keras.layers.MaxPooling2D(pool_size=2)(x)
    
    x,nfilter = dense_block(x,growth_rate,nfilter,6)
    x,nfilter = transition_block(x,nfilter)
    x,nfilter = dense_block(x,growth_rate,nfilter,12)
    x,nfilter = transition_block(x,nfilter)
    x,nfilter = dense_block(x,growth_rate,nfilter,24)
    x,nfilter = transition_block(x,nfilter)
    x,nfilter = dense_block(x,growth_rate,nfilter,16)
    
#     x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(361,activation='softmax')(x)
    model = keras.Model(inputs=inputs,outputs=x)
    return model

In [11]:
model = get_model()
opt = Nadam(learning_rate = 0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

224
496
1016
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 19, 19, 11)] 0                                            
__________________________________________________________________________________________________
conv2d_120 (Conv2D)             (None, 10, 10, 32)   17280       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 10, 10, 32)   128         conv2d_120[0][0]                 
__________________________________________________________________________________________________
activation_119 (Activation)     (None, 10, 10, 32)   0           batch_normalization_120[0][0]    
_______________________________________________________________________________

In [81]:
def modified_model(img_shape, num_classes):
    model_resnet = ResNet50(include_top=False, weights=None)

    img_input = Input(shape=img_shape)
    img_model_resnet = model_resnet(img_input)

    x = Flatten(name = 'flatten')(img_model_resnet)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    modified_model = Model(inputs=img_input, outputs=x)
    opt = Nadam(learning_rate = 0.001)
    modified_model.compile(loss='categorical_crossentropy',
                            optimizer=opt, metrics=['acc'])

    return modified_model

In [85]:
model = modified_model((19, 19, 11), 1)

ValueError: Input 0 of layer conv1_conv is incompatible with the layer: expected axis -1 of input shape to have value 3 but received input with shape (None, 25, 25, 11)

In [32]:
model = load_model('./models/model_kyu_10_14.h5')

In [62]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset,
    shuffle = True
)

['B[pd]', 'W[dp]', 'B[pq]', 'W[dd]', 'B[fq]', 'W[nd]', 'B[nc]', 'W[mc]', 'B[oc]', 'W[md]', 'B[qf]', 'W[iq]', 'B[cn]', 'W[fp]', 'B[cq]', 'W[cp]', 'B[dq]', 'W[eq]', 'B[er]', 'W[ep]', 'B[bp]', 'W[fr]', 'B[bo]', 'W[dr]', 'B[cr]', 'W[es]', 'B[dj]', 'W[po]', 'B[qo]', 'W[pp]', 'B[qp]', 'W[oq]', 'B[qq]', 'W[np]', 'B[pn]', 'W[on]', 'B[pm]', 'W[om]', 'B[cf]', 'W[pl]', 'B[ql]', 'W[qm]', 'B[qn]', 'W[qk]', 'B[rm]', 'W[rl]', 'B[qm]', 'W[ok]', 'B[fc]', 'W[df]', 'B[dg]', 'W[ef]', 'B[eg]', 'W[ce]', 'B[bf]', 'W[fg]', 'B[db]', 'W[cc]', 'B[cb]', 'W[bb]', 'B[ic]', 'W[gd]', 'B[gc]', 'W[hd]', 'B[hc]', 'W[id]', 'B[jd]', 'W[je]', 'B[kd]', 'W[ke]', 'B[qi]', 'W[ld]', 'B[fh]', 'W[kc]', 'B[jc]', 'W[kb]', 'B[jb]', 'W[gh]', 'B[fi]', 'W[gi]', 'B[km]', 'W[fj]', 'B[ej]', 'W[fk]', 'B[dl]', 'W[em]', 'B[kp]', 'W[dm]', 'B[cm]', 'W[el]', 'B[dk]', 'W[lo]', 'B[ko]', 'W[ln]', 'B[kn]', 'W[lp]', 'B[lm]', 'W[kq]', 'B[lj]', 'W[jp]', 'B[hn]', 'W[io]', 'B[in]', 'W[go]', 'B[hk]', 'W[gk]', 'B[hj]', 'W[gj]', 'B[ii]', 'W[hl]', 'B[il]', 

      3/Unknown - 3s 92ms/step - loss: 6.8218 - accuracy: 0.0052['B[pd]', 'W[dp]', 'B[pp]', 'W[dd]', 'B[pj]', 'W[nc]', 'B[nd]', 'W[md]', 'B[ne]', 'W[oc]', 'B[pc]', 'W[lc]', 'B[pf]', 'W[nq]', 'B[np]', 'W[mp]', 'B[no]', 'W[oq]', 'B[pq]', 'W[op]', 'B[oo]', 'W[lq]', 'B[pn]', 'W[ln]', 'B[cn]', 'W[fp]', 'B[dj]', 'W[cg]', 'B[fc]', 'W[ec]', 'B[fd]', 'W[ic]', 'B[cc]', 'W[eb]', 'B[ce]', 'W[de]', 'B[bf]', 'W[cf]', 'B[cd]', 'W[bg]', 'B[ae]', 'W[ef]', 'B[cb]', 'W[gf]', 'B[bp]', 'W[cq]', 'B[bq]', 'W[br]', 'B[cp]', 'W[dq]', 'B[do]', 'W[eo]', 'B[en]', 'W[fn]', 'B[em]', 'W[fm]', 'B[jm]', 'W[ll]', 'B[jk]', 'W[jo]', 'B[hn]', 'W[ho]', 'B[go]', 'W[gn]', 'B[io]', 'W[hp]', 'B[in]', 'W[ip]', 'B[ji]', 'W[lj]', 'B[fk]', 'W[fl]', 'B[el]', 'W[gk]', 'B[fj]', 'W[gj]', 'B[gi]', 'W[hi]', 'B[jg]', 'W[gh]', 'B[lf]', 'W[fi]', 'B[bi]', 'W[ci]', 'B[cj]', 'W[di]', 'B[ei]', 'W[eh]', 'B[ej]', 'W[bh]', 'B[bj]', 'W[kh]', 'B[jh]', 'W[kg]', 'B[kf]', 'W[jf]', 'B[ki]', 'W[li]', 'B[lh]', 'W[mh]', 'B[lg]', 'W[ni]', 'B[kk]', 'W[lk]',

      7/Unknown - 3s 88ms/step - loss: 6.6609 - accuracy: 0.0112['B[pd]', 'W[dp]', 'B[pp]', 'W[dc]', 'B[pj]', 'W[nc]', 'B[nd]', 'W[md]', 'B[ne]', 'W[oc]', 'B[pc]', 'W[ic]', 'B[de]', 'W[dh]', 'B[cc]', 'W[dd]', 'B[cd]', 'W[ee]', 'B[df]', 'W[ef]', 'B[dg]', 'W[eg]', 'B[db]', 'W[eb]', 'B[cb]', 'W[fc]', 'B[ch]', 'W[ci]', 'B[cg]', 'W[bi]', 'B[bh]', 'W[ei]', 'B[jd]', 'W[id]', 'B[je]', 'W[kc]', 'B[ie]', 'W[he]', 'B[hf]', 'W[gf]', 'B[jc]', 'W[jb]', 'B[ib]', 'W[hb]', 'B[hd]', 'W[ia]', 'B[ge]', 'W[fe]', 'B[hc]', 'W[ib]', 'B[gg]', 'W[fg]', 'B[ig]', 'W[gd]', 'B[he]', 'W[gc]', 'B[gh]', 'W[jq]', 'B[cn]', 'W[co]', 'B[dn]', 'W[fp]', 'B[ck]', 'W[bo]', 'B[bn]', 'W[an]', 'B[am]', 'W[ao]', 'B[bl]', 'W[dj]', 'B[fn]', 'W[ho]', 'B[fk]', 'W[gi]', 'B[hi]', 'W[gj]', 'B[gk]', 'W[hj]', 'B[ii]', 'W[ij]', 'B[ji]', 'W[ej]', 'B[dk]', 'W[ek]', 'B[el]', 'W[jj]', 'B[ki]', 'W[nq]', 'B[oq]', 'W[np]', 'B[pn]', 'W[nn]', 'B[hn]', 'W[in]', 'B[hm]', 'W[im]', 'B[go]', 'W[gp]', 'B[nr]', 'W[mr]', 'B[or]', 'W[lq]', 'B[pg]', 'W[ob]',

     11/Unknown - 3s 85ms/step - loss: 6.4983 - accuracy: 0.0142['B[dd]', 'W[pq]', 'B[po]', 'W[qo]', 'B[qn]', 'W[qp]', 'B[pn]', 'W[nq]', 'B[qj]', 'W[jp]', 'B[qf]', 'W[qe]', 'B[pf]', 'W[nc]', 'B[cn]', 'W[co]', 'B[dn]', 'W[fq]', 'B[dj]', 'W[fc]', 'B[hc]', 'W[cc]', 'B[dc]', 'W[cd]', 'B[de]', 'W[db]', 'B[eb]', 'W[cb]', 'B[ec]', 'W[cf]', 'B[ce]', 'W[be]', 'B[df]', 'W[cg]', 'B[dg]', 'W[ch]', 'B[dh]', 'W[id]', 'B[hd]', 'W[if]', 'B[ic]', 'W[he]', 'B[jd]', 'W[ie]', 'B[fd]', 'W[kg]', 'B[ci]', 'W[bi]', 'B[bj]', 'W[bh]', 'B[lc]', 'W[kj]', 'B[oj]', 'W[fn]', 'B[eo]', 'W[fo]', 'B[ep]', 'W[eq]', 'B[fp]', 'W[gp]', 'B[gq]', 'W[hq]', 'B[gr]', 'W[hr]', 'B[dq]', 'W[fr]', 'B[cp]', 'W[dr]', 'B[cq]', 'W[cr]', 'B[br]', 'W[gs]', 'B[qc]', 'W[pc]', 'B[re]', 'W[qd]', 'B[rd]', 'W[rc]', 'B[qb]', 'W[rb]', 'B[pb]', 'W[ob]', 'B[ra]', 'W[pa]', 'B[sb]', 'W[qa]', 'B[sc]', 'W[sa]', 'B[sd]', 'W[rn]', 'B[rm]', 'W[ro]', 'B[ql]', 'W[oh]', 'B[ph]', 'W[pi]', 'B[qh]', 'W[pj]', 'B[pk]', 'W[oi]', 'B[nj]', 'W[mh]', 'B[og]', 'W[ng]',

     14/Unknown - 4s 85ms/step - loss: 6.3925 - accuracy: 0.0151['B[dd]', 'W[pq]', 'B[po]', 'W[qo]', 'B[qn]', 'W[qp]', 'B[pn]', 'W[nq]', 'B[qj]', 'W[qh]', 'B[oj]', 'W[cf]', 'B[fc]', 'W[bd]', 'B[cc]', 'W[ci]', 'B[cn]', 'W[co]', 'B[dn]', 'W[fq]', 'B[ck]', 'W[dj]', 'B[dk]', 'W[kc]', 'B[qc]', 'W[pc]', 'B[qd]', 'W[qe]', 'B[re]', 'W[qf]', 'B[rf]', 'W[rg]', 'B[pb]', 'W[ob]', 'B[qb]', 'W[nc]', 'B[ic]', 'W[ip]', 'B[kp]', 'W[kq]', 'B[lq]', 'W[jq]', 'B[mp]', 'W[rn]', 'B[rm]', 'W[ro]', 'B[ql]', 'W[op]', 'B[no]', 'W[oo]', 'B[on]', 'W[oh]', 'B[mj]', 'W[mh]', 'B[kj]', 'W[kh]', 'B[je]', 'W[ke]', 'B[kf]', 'W[lf]', 'B[kd]', 'W[le]', 'B[ld]', 'W[md]', 'B[lc]', 'W[jf]', 'B[ie]', 'W[mc]', 'B[lb]', 'W[km]', 'B[jn]', 'W[jl]', 'B[hn]', 'W[hl]', 'B[fn]', 'W[hj]', 'B[ej]', 'W[ei]', 'B[fj]', 'W[fi]', 'B[gj]', 'W[gi]', 'B[hi]', 'W[ii]', 'B[hh]', 'W[jj]', 'B[hk]', 'W[jk]', 'B[ij]', 'W[ji]', 'B[gl]', 'W[mm]', 'B[nm]', 'W[nl]', 'B[ol]', 'W[nk]', 'B[nj]', 'W[ll]', 'B[ok]', 'W[bn]', 'B[bm]', 'W[bo]', 'B[bc]', 'W[cd]',

...

     31/Unknown - 5s 85ms/step - loss: 5.9768 - accuracy: 0.0197['B[pd]', 'W[dp]', 'B[pq]', 'W[dd]', 'B[qk]', 'W[nd]', 'B[ne]', 'W[me]', 'B[nf]', 'W[mf]', 'B[od]', 'W[nc]', 'B[ng]', 'W[mg]', 'B[fc]', 'W[nh]', 'B[ph]', 'W[og]', 'B[oe]', 'W[oh]', 'B[pg]', 'W[pi]', 'B[qi]', 'W[pj]', 'B[qj]', 'W[pk]', 'B[ic]', 'W[pf]', 'B[qf]', 'W[qg]', 'B[qh]', 'W[qe]', 'B[rf]', 'W[pe]', 'B[re]', 'W[qd]', 'B[qc]', 'W[rd]', 'B[rc]', 'W[of]', 'B[pc]', 'W[pl]', 'B[lc]', 'W[md]', 'B[rg]', 'W[ql]', 'B[sd]', 'W[lh]', 'B[db]', 'W[fd]', 'B[gd]', 'W[ec]', 'B[fb]', 'W[ge]', 'B[hd]', 'W[ff]', 'B[eb]', 'W[dc]', 'B[cc]', 'W[cd]', 'B[bc]', 'W[bd]', 'B[cn]', 'W[nq]', 'B[qo]', 'W[oo]', 'B[iq]', 'W[po]', 'B[pp]', 'W[oq]', 'B[pr]', 'W[mo]', 'B[fq]', 'W[fp]', 'B[gp]', 'W[fo]', 'B[ck]', 'W[eq]', 'B[fr]', 'W[er]', 'B[lq]', 'W[qn]', 'B[ro]', 'W[rn]', 'B[rq]', 'W[bo]', 'B[bn]', 'W[co]', 'B[ch]', 'W[dn]', 'B[dm]', 'W[em]', 'B[dl]', 'W[dg]', 'B[ko]', 'W[hn]', 'B[km]', 'W[jl]', 'B[ll]', 'W[jn]', 'B[kn]', 'W[el]', 'B[ek]', 'W[fk]',

     35/Unknown - 5s 85ms/step - loss: 5.9170 - accuracy: 0.0208['B[pd]', 'W[dp]', 'B[pq]', 'W[dd]', 'B[qk]', 'W[dj]', 'B[cq]', 'W[cp]', 'B[dq]', 'W[eq]', 'B[er]', 'W[fq]', 'B[fr]', 'W[gq]', 'B[gr]', 'W[hq]', 'B[ek]', 'W[dm]', 'B[dk]', 'W[nc]', 'B[nd]', 'W[md]', 'B[ne]', 'W[oc]', 'B[pc]', 'W[kc]', 'B[pg]', 'W[gc]', 'B[cc]', 'W[dc]', 'B[cd]', 'W[ce]', 'B[be]', 'W[cf]', 'B[bf]', 'W[cg]', 'B[db]', 'W[eb]', 'B[cb]', 'W[fc]', 'B[ej]', 'W[cj]', 'B[eh]', 'W[qi]', 'B[qh]', 'W[pi]', 'B[qn]', 'W[qp]', 'B[rp]', 'W[qq]', 'B[rq]', 'W[pp]', 'B[qr]', 'W[oq]', 'B[pr]', 'W[np]', 'B[kq]', 'W[nn]', 'B[ko]', 'W[pk]', 'B[ql]', 'W[ol]', 'B[km]', 'W[fm]', 'B[gk]', 'W[hm]', 'B[ik]', 'W[ck]', 'B[jl]', 'W[lf]', 'B[if]', 'W[id]', 'B[je]', 'W[gg]', 'B[ih]', 'W[me]', 'B[nf]', 'W[hf]', 'B[hg]', 'W[gf]', 'B[gh]', 'W[ie]', 'B[jf]', 'W[ef]', 'B[bp]', 'W[bo]', 'B[bq]', 'W[cn]', 'B[pb]', 'W[ob]', 'B[bg]', 'W[ch]', 'B[rh]', 'W[ri]', 'B[rj]', 'W[si]', 'B[sh]', 'W[qj]', 'B[rk]', 'W[pa]', 'B[qa]', 'W[oa]', 'B[rb]', 'W[oh]',

     39/Unknown - 6s 85ms/step - loss: 5.8757 - accuracy: 0.0218['B[pd]', 'W[pp]', 'B[cp]', 'W[fc]', 'B[dc]', 'W[ce]', 'B[cd]', 'W[de]', 'B[be]', 'W[cg]', 'B[bc]', 'W[ic]', 'B[qf]', 'W[cj]', 'B[eq]', 'W[cm]', 'B[qn]', 'W[nq]', 'B[rp]', 'W[qq]', 'B[qk]', 'W[jp]', 'B[hq]', 'W[ho]', 'B[jj]', 'W[eo]', 'B[gj]', 'W[qi]', 'B[rj]', 'W[ok]', 'B[pj]', 'W[ni]', 'B[oi]', 'W[nh]', 'B[oh]', 'W[oj]', 'B[pi]', 'W[ng]', 'B[mj]', 'W[om]', 'B[mm]', 'W[lk]', 'B[lj]', 'W[kk]', 'B[kj]', 'W[mk]', 'B[jk]', 'W[km]', 'B[im]', 'W[hl]', 'B[il]', 'W[hk]', 'B[hj]', 'W[ik]', 'B[jl]', 'W[ij]', 'B[ii]', 'W[fl]', 'B[jh]', 'W[hh]', 'B[hi]', 'W[jg]', 'B[gh]', 'W[hg]', 'B[kg]', 'W[kf]', 'B[jf]', 'W[ig]', 'B[lf]', 'W[kh]', 'B[ke]', 'W[lh]', 'B[mh]', 'W[mg]', 'B[mi]', 'W[lg]', 'B[gf]', 'W[gg]', 'B[fg]', 'W[fh]', 'B[gi]', 'W[ff]', 'B[eg]', 'W[hf]', 'B[ge]', 'W[he]', 'B[ei]', 'W[ef]', 'B[fk]', 'W[hm]', 'B[eh]', 'W[ek]', 'B[ej]', 'W[gk]', 'B[fj]', 'W[dk]', 'B[jn]', 'W[lm]', 'B[ko]', 'W[kq]', 'B[mo]', 'W[lp]', 'B[lo]', 'W[nn]',

KeyboardInterrupt: 

['B[pd]', 'W[dp]', 'B[pq]', 'W[dd]', 'B[fq]', 'W[cn]', 'B[qk]', 'W[nc]', 'B[nd]', 'W[md]', 'B[ne]', 'W[oc]', 'B[pc]', 'W[me]', 'B[nf]', 'W[mf]', 'B[hc]', 'W[jc]', 'B[dc]', 'W[cc]', 'B[ec]', 'W[cd]', 'B[ed]', 'W[ee]', 'B[fe]', 'W[ef]', 'B[cb]', 'W[bb]', 'B[db]', 'W[ff]', 'B[ge]', 'W[gf]', 'B[lc]', 'W[mc]', 'B[kc]', 'W[jd]', 'B[jb]', 'W[ib]', 'B[ic]', 'W[kb]', 'B[hb]', 'W[ja]', 'B[he]', 'W[hf]', 'B[id]', 'W[je]', 'B[ie]', 'W[if]', 'B[pg]', 'W[po]', 'B[qo]', 'W[qn]', 'B[qp]', 'W[pm]', 'B[nq]', 'W[pk]', 'B[qj]', 'W[ql]', 'B[pj]', 'W[ok]', 'B[oj]', 'W[nk]', 'B[iq]', 'W[lq]', 'B[dr]', 'W[cq]', 'B[ci]', 'W[ck]', 'B[cf]', 'W[de]', 'B[ei]', 'W[dh]', 'B[di]', 'W[ch]', 'B[bh]', 'W[bg]', 'B[bi]', 'W[cg]', 'B[ek]', 'W[em]', 'B[gl]', 'W[gm]', 'B[hm]', 'W[gn]', 'B[hk]', 'W[hn]', 'B[im]', 'W[in]', 'B[fl]', 'W[jm]', 'B[jl]', 'W[km]', 'B[jj]', 'W[kl]', 'B[jk]', 'W[jp]', 'B[jr]', 'W[kr]', 'B[pb]', 'W[jq]', 'B[js]', 'W[ks]', 'B[hr]', 'W[cr]', 'B[cs]', 'W[er]', 'B[eq]', 'W[dq]', 'B[ds]', 'W[bs]', 'B[fr]', 

In [ ]:
model.save('./models/model_kyu_10_15_f64_allk3_12layerC.h5')

In [28]:
history = model.fit(
    dataset,
    epochs = 1
)

105995/105995 [==============================] - 15947s 150ms/step - loss: 1.7524 - accuracy: 0.5309


In [29]:
model.save('./models/model_kyu_10_15_f64.h5')

In [30]:
history = model.fit(
    dataset,
    epochs = 1
)

105995/105995 [==============================] - 15979s 151ms/step - loss: 1.6064 - accuracy: 0.5625


In [31]:
model.save('./models/model_kyu_10_15_f128_2.h5')

In [32]:
history = model.fit(
    dataset,
    epochs = 1
)

     38/Unknown - 6s 153ms/step - loss: 1.4566 - accuracy: 0.6145

KeyboardInterrupt: 

In [ ]:
model.save('./models/model_kyu_10_15_f128_3.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_4.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_5.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_6.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_7.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_8.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_9.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_14_11.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!